In [ ]:
# Установка необходимых библиотек
!pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO

# Отключение интеграции с wandb
os.environ['WANDB_DISABLED'] = 'true'

# Загрузка модели YOLOv8
model = YOLO('yolov8m.pt')

# Обучение модели
# Замените '/kaggle/input/potato-colorado/data.yaml' на путь к data.yaml
# epochs - количество эпох для обучения
# imgsz - размер изображений для обучения
model.train(data='/kaggle/input/potato-colorado/data.yaml', epochs=10, imgsz=640)

# Проверка модели на тестовых данных
results = model.val()

# Вывод результатов проверки
print(results)


In [ ]:
import cv2
import matplotlib.pyplot as plt

def to_numpy(tensor):
    return tensor.cpu().numpy() if tensor.is_cuda else tensor.numpy()

# Использование модели для предсказания на изображении - укажите путь к своему изображению
results = model.predict(source='/kaggle/input/colorado/photo_2024-06-19_10-23-25.jpg', save=False)

# Получение результатов
for result in results:
    img = result.orig_img
    boxes = to_numpy(result.boxes.xyxy)  # Перемещение тензора на CPU и преобразование в numpy array
    confidences = to_numpy(result.boxes.conf)  # Получение уверенности
    class_ids = to_numpy(result.boxes.cls)  # Получение идентификаторов классов
    names = result.names
    
    # Отрисовка рамок на изображении
    for box, confidence, class_id in zip(boxes, confidences, class_ids):
        x1, y1, x2, y2 = map(int, box[:4])
        label = names[int(class_id)]
        
        # Рисование рамки и текста на изображении
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Показ изображения с рамками
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
